In [1]:
pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 8.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 9.5 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [11]:
import csv
import pandas as pd

Define error correction

In [68]:
import difflib

valid_types = [
    'normal',
    'fire',
    'water',
    'grass',
    'electric',
    'ice',
    'poison',
    'flying',
    'psychic',
    'bug',
    'fighting',
    'rock',
    'ghost',
    'dark',
    'dragon',
    'steel',
    'fairy',
    'none',
    'ground'
]

def validate_type(type):
    if type not in valid_types:
        print (type)
    return type
        

Process data from csv

In [69]:
import csv

headers = []
datalist = []
# nameset = set()

with open('Pokemon_full_2.csv') as csvpoke:
    reader = csv.reader(csvpoke, delimiter=',')
    headers = next(reader)

    type_1_index = headers.index('type')
    type_2_index = headers.index('secundary type')

    for row in reader:
        rowData = [elem.lower().strip() for elem in row]
            
        rowData[type_1_index] = validate_type(rowData[type_1_index])
        rowData[type_2_index] = validate_type(rowData[type_2_index])
        
        if '' not in row:
            datalist.append(rowData)

datalist


snake
snake
jesusmon
gooff
jesusmon
mon


[['bulbasaur',
  '7',
  '69',
  'grass',
  'poison',
  '45',
  '49',
  '49',
  '65',
  '65',
  '45'],
 ['fushigisou',
  '10',
  '130',
  'grass',
  'poison',
  '60',
  '62',
  '63',
  '80',
  '80',
  '60'],
 ['ivysaur',
  '10',
  '130',
  'grass',
  'poison',
  '60',
  '62',
  '63',
  '80',
  '80',
  '60'],
 ['venusaur',
  '20',
  '1000',
  'grass',
  'poison',
  '80',
  '82',
  '83',
  '100',
  '100',
  '80'],
 ['charmander', '6', '85', 'fire', 'none', '39', '52', '43', '60', '50', '65'],
 ['charmeleon',
  '11',
  '190',
  'fire',
  'none',
  '58',
  '64',
  '58',
  '80',
  '65',
  '80'],
 ['charizard',
  '17',
  '905',
  'fire',
  'flying',
  '78',
  '84',
  '78',
  '109',
  '85',
  '100'],
 ['squirtle', '5', '90', 'water', 'none', '44', '48', '65', '50', '64', '43'],
 ['wartortle',
  '10',
  '225',
  'water',
  'none',
  '59',
  '63',
  '80',
  '65',
  '80',
  '58'],
 ['blastoise',
  '16',
  '855',
  'water',
  'none',
  '79',
  '83',
  '100',
  '85',
  '105',
  '78'],
 ['caterpie',

Create pandas dataframe

In [50]:
pokedf = pd.DataFrame(datalist[1:], columns=datalist[0])

pokedf


,name,height,weight,type,secundary type,hp,attack,defense,sp atk,sp def,speed
0,bulbasaur,7,69,grass,poison,45,49,49,65,65,45
1,fushigisou,10,130,grass,poison,60,62,63,80,80,60
2,ivysaur,10,130,grass,poison,60,62,63,80,80,60
3,venusaur,20,1000,grass,poison,80,82,83,100,100,80
4,charmander,6,85,fire,none,39,52,43,60,50,65
...,...,...,...,...,...,...,...,...,...,...,...
891,drakloak,14,110,dragon,ghost,68,80,50,60,50,102
892,dragapult,30,500,dragon,ghost,88,120,75,100,75,142
893,zacian,28,1100,fairy,none,92,130,115,80,115,138
894,zamazenta,29,2100,fighting,none,92,130,115,80,115,138


Drop duplicate lines

In [52]:
pokedf = pokedf.drop_duplicates()

pokedf

,name,height,weight,type,secundary type,hp,attack,defense,sp atk,sp def,speed
0,bulbasaur,7,69,grass,poison,45,49,49,65,65,45
1,fushigisou,10,130,grass,poison,60,62,63,80,80,60
2,ivysaur,10,130,grass,poison,60,62,63,80,80,60
3,venusaur,20,1000,grass,poison,80,82,83,100,100,80
4,charmander,6,85,fire,none,39,52,43,60,50,65
...,...,...,...,...,...,...,...,...,...,...,...
891,drakloak,14,110,dragon,ghost,68,80,50,60,50,102
892,dragapult,30,500,dragon,ghost,88,120,75,100,75,142
893,zacian,28,1100,fairy,none,92,130,115,80,115,138
894,zamazenta,29,2100,fighting,none,92,130,115,80,115,138
